# Concept extraction from text
  
    

## 1. Loading text file into string 

### Option 1. Downloading a wikipedia article's text

In [1]:
from bs4 import BeautifulSoup
import requests
url = 'https://en.wikipedia.org/wiki/Human_digestive_system'

source = requests.get(url).text
soup = BeautifulSoup(source,'lxml')


text_set = soup.find_all(['p']) ## This will skip headings ('h2','h3') and lists that are made as links( 'li')
text_list = [p1.get_text() for p1 in text_set]
tags_list = [p1.name for p1 in text_set ]

rawtxt = ''.join(text_list)

print("length of material")
print(len(rawtxt))

print("Sample of text")
print(rawtxt[0:500])

length of material
40417
Sample of text

The human digestive system consists of the gastrointestinal tract plus the accessory organs of digestion (the tongue, salivary glands, pancreas, liver, and gallbladder). Digestion involves the breakdown of food into smaller and smaller components, until they can be absorbed and assimilated into the body. The process of digestion has three stages. The first stage is the cephalic phase of digestion which begins with gastric secretions in response to the sight and smell of food. This stage include


#### Save rawtxt as is for later:

In [2]:
#filename = 'A Mind For Numbers_ How to Excel at Math and Science (Even If You Flunked Algebra).txt'
#path_name = "C:/Users/Arati/Documents/personal docs/cognitive science and education/"
# path_name = "C:/Users/Arati/Documents/personal docs/python_introduction_course/textdata/"
# path_name = "C:/Users/Arati/Documents/personal docs/cognitive science and education/"
#filename = 'Physics  for IIT- JEE (Mains & Advanced)  Vol. 1 of 4_  Complete Study Pack of Physics_ Mechanics  for Engineering Entrance Examination (Sachan Book 62).txt'
path_name = 'C:/Users/Arati/Conceptmap project/IntelligentTutoringSystem_Experiments/processed_data'
filename = 'Wiki_human_digestive_system'
with open(path_name + filename,"a",encoding="utf-8") as myfile:
    myfile.write(rawtxt)
myfile.close()

### Option 2. Getting file from disk:

In [1]:
fnames = ['Complex PTSD_ From Surviving to Thriving_ A GUIDE AND MAP FOR RECOVERING FROM CHILDHOOD TRAUMA.txt']
# path_name = "C:/Users/Arati/Documents/personal docs/python_introduction_course/textdata/"
path_name = "C:/Users/Arati/Conceptmap project/IntelligentTutoringSystem_Experiments/raw_data/"
rawtxt = []
for f in fnames:
    with open(path_name + f, "r",encoding = "utf-8") as myfile:
        rawtxt.append(myfile.read())
    print(f)
for t in rawtxt:
    print(len(t))
rawtxt = ('\n').join(rawtxt)
len(rawtxt)

Complex PTSD_ From Surviving to Thriving_ A GUIDE AND MAP FOR RECOVERING FROM CHILDHOOD TRAUMA.txt
563960


563960

In [1]:
filename = 'wikihow_1'
path_name = "C:/Users/Arati/Documents/personal docs/python_introduction_course/textdata/"
with open (path_name +filename+'.txt', "r",encoding="utf-8") as myfile:
    rawtxt0=myfile.read()

In [2]:
filename = 'wikipedia_4'
path_name = "C:/Users/Arati/Documents/personal docs/python_introduction_course/textdata/"
with open (path_name +filename+'.txt', "r",encoding="utf-8") as myfile:
    rawtxt1=myfile.read()

In [3]:
rawtxt = rawtxt0+rawtxt1

#rawtxt = rawtxt.encode('ascii','ignore')

In [1]:
rawtxt = 'Classical mechanics describes the motion of macroscopic objects, from projectiles to parts of machinery, and astronomical objects, such as spacecraft, planets, stars and galaxies.'

In [3]:
for letter in 'abcdefghijklmnopqrstuvwxyz':
    try:
        print(wnl.lemmatize('am',letter))
    except:
        print(letter+' not a key')

a not a key
b not a key
c not a key
d not a key
e not a key
f not a key
g not a key
h not a key
i not a key
j not a key
k not a key
l not a key
m not a key
n not a key
o not a key
p not a key
q not a key
r not a key
s not a key
t not a key
u not a key
v not a key
w not a key
x not a key
y not a key
z not a key


## Extracting list of concepts:

### Importing libraries and modules

In [2]:
from nltk import word_tokenize
from nltk.chunk import *
from nltk.chunk.util import *
from nltk.chunk.regexp import *
from nltk import Tree
import re
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))
import nltk
wnl = nltk.WordNetLemmatizer()
porter = nltk.PorterStemmer()
lancaster = nltk.LancasterStemmer()

In [3]:
with open ('../processed_data/stopconcept_list.txt', "r",encoding="utf-8") as myfile:
    stop_concepts=myfile.read()
stop_concepts = stop_concepts.split(', ')

In [6]:
stop_concepts = []

### Sentence splitting
#### Option 1: split on periods

In [7]:
sents = rawtxt.split('.')

#### Option 2: Pretrained NLTK sentence splitter

In [8]:
sents = nltk.sent_tokenize(rawtxt)

#### Option 2: Train unsupervised on domain text

In [10]:
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(rawtxt)
tokenizer = PunktSentenceTokenizer(trainer.get_params())

In [11]:
sents = tokenizer.tokenize(rawtxt)

In [12]:
print("Number of sentences in text "+str(len(sents)))
print(len(sents))

print("Sample of sentences:")
print(sents[0:20])

Number of sentences in text 3330
3330
Sample of sentences:
['Title : Complex PTSD: From Surviving to Thriving: A GUIDE AND MAP FOR RECOVERING FROM CHILDHOOD TRAUMA\nAuthor: Walker, Pete\n\nGenerate From: eCore v0.9.12.751 [ http://www.epubor.com/ecore.html ]\n\nTechnology Website: http://www.epubor.com\n\n\n\n [image file=image8.jpg]  [image file=image17.jpg] Testimonials about Pete Walker’s first book, The Tao of Fully Feeling, and his website: www.pete-walker.comI am writing from Survivors of Abuse Recovering (S.O.A.R.) Society, located in Canada.', 'We would like to include “13 Steps for Managing Flashbacks” in our resource manual.I found myself.', 'I found myself in your words.', 'It’s as if you had unzipped me, stepped inside my traumatized inner self, meandered around a bit, come back outside, and wrote about what you discovered inside of me.', 'For the first time in my life.......and I’m in my fifties now........I don’t feel defective......or crazy.......or “weird”.......or even

### Token handling functions: 
1. validchar(wrd): checks if token is a valid alphanumeric+hyphens word
2. lemmatize_by_pos(tag) lemmatizes token by part of speech
3. chunk_this(grammar_rule_key,sentence_tags) chunks a particular grammar rule key (see chunkrules)
4. eqn_label: extracts equation terms and replaces all occurences in text with a textkey, which is then treated as a noun phrase. Also updates equation dictionary
5. display_equation (displays equation term by key)
6. chunker: chunks each sentence by each chunking rule

In [13]:
def validchar(wrd):
    p = re.compile(r'[^0-9a-zA-Z_-]')
    if p.search(wrd) is None:
        return 1
    else:
        return 0

    
# wdlist = {'be','can','could','should','as','would','must','ought','will','i',"won't","can't","isn't","shan't",'to','the','a','an',
#           'of','at','by','if','use', 'shall','about','around','and','or','try','he','she','this','they','them','these','those',
#           'it','for','useful','not',"it's",'its','with','which','that','than'}
wdlist = {}
def lemmatize_by_pos(tag,wdlist):
    token = tag[0].lower()
    pos = tag[1]
#     if token in stop_words:
#         return (token,pos)
    if pos.startswith('J'):
        # adjective form
        lemma = wnl.lemmatize(token,'s')
    elif pos.startswith('N'):
        # noun form
        lemma = wnl.lemmatize(token,'n')
    elif pos.startswith('R'):
        # adverb
        lemma = wnl.lemmatize(token,'r')
    elif pos.startswith('V'):
        lemma = wnl.lemmatize(token,'v')
    else:
        lemma = token
    if lemma in wdlist:
        pos = lemma
    return (lemma,pos)





global eqn_dict
eqn_dict = {}
global eqn_count
eqn_count = 1

def eqn_label(tokens):
    global eqn_count
    global eqn_dict
    EQNlist = [wrd for wrd in tokens if not(wrd.isalnum()) and re.search(r'[\[\]\{\}\+*^=_%$]',wrd) and len(wrd)>1 ]
    ## replace queations with a label and save to equation dictionary
    for eqn in EQNlist:
        
        if not(eqn in eqn_dict):
            
            eqn_dict[eqn] = ''.join(['equation',str(eqn_count)])
            eqn_count = eqn_count + 1                          
        else:    
            tokens[tokens.index(eqn)] = eqn_dict[eqn]
                  
    return tokens

global inv_eqn_dict
inv_eqn_dict = dict([[value,key] for key,value in eqn_dict.items()])

def display_equation(reptokens):
    for wrd in reptokens:
        if wrd in inv_eqn_dict:
            reptokens[reptokens.index(wrd)] = inv_eqn_dict[wrd]
    return reptokens

Setting up chunking rules:

Chunking done in batches to enable overlapping tokens to be extracted. 

In [14]:
chunkrules = {}

# Define chunking rules here:
chunkrules['JJNP'] = r"""    
    JJNP: {<RB.*>?<J.*>?<NN.*>{1,}}       
"""
## Examples: "reusable contactless stored value smart card"

def chunk_this(grammar_rule_key,sentence_tags):
    setlist = []
    cp = nltk.RegexpParser(chunkrules[grammar_rule_key])
    J = cp.parse(sentence_tags) 
    for i in range(len(J)):
        if not(isinstance(J[i],tuple)):
            if (J[i].label()==grammar_rule_key):
                setlist.append((' '.join([J[i][j][0] for j in range(len(J[i])) if (validchar(J[i][j][0])==1)])))
    setlist = list(set(setlist))
    setlist = [wrd.lower() for wrd in setlist if (len(wrd)>1 and wrd not in stop_concepts)]
    return setlist

def chunker(sentence_tags):
    return [chunk_this(key,sentence_tags)  for key in chunkrules]


Process each sentence:

In [15]:
%%time 
sent_to_np = {}
sent_to_ltags = {}
sent_to_tags = {}

for i in range(len(sents)):
    tokens = word_tokenize(sents[i])
    reptokens = eqn_label(tokens)
    tags = nltk.pos_tag(reptokens)
    lemmatags = [lemmatize_by_pos(t,wdlist) for t in tags]
    sent_to_np[i] = chunker(lemmatags)
    sent_to_ltags[i] = lemmatags
    sent_to_tags[i] = tags

Wall time: 13.7 s


In [17]:
sent_to_np[400]

[['definition',
  'developmental task',
  'pre-verbal child',
  '2-3 year-olds',
  'feeling']]

In [18]:
# Flatten lists of lists containing chunks of different rules, dictionary of np to sent
import itertools
sent_to_npflat = {}
np_to_sent = {}
for key in sent_to_np:
    sent_to_npflat[key] = list(set((itertools.chain(*sent_to_np[key]))))  
    for np in sent_to_npflat[key]:            
        if np in np_to_sent:                           
            np_to_sent[np].append(key)
        else:                
            np_to_sent[np]=[key]

### Create dataframe with some metrics:
- Concept: concept phrase
- Occurence: list of sentences in which the phrase occurs
- Frequency: number of sentences in which the phrase occurs
- Mean: average of sentence numbers in the text in which the phrase occurs normalized to number of sentences
- Median: median of sentence numbers in the text in which the phrase occurs normalized to number of sentences. Lets us know if phrase occurs much more in the beginning of the text, or towards the end. can indicate how central the phrase is to the text. 
- Sdev: standard deviation of the sentences in which the phrase occurs (indicates the dispersion of the phrase in the text)

In [19]:
import numpy as num
import pandas as pd
Concept = pd.Series([key for (key,value) in np_to_sent.items()])
Occurence = pd.Series([num.array(value) for (key,value) in np_to_sent.items()])
Frequency = pd.Series([len(o) for o in Occurence])
Mean= pd.Series([num.mean(o) for o in Occurence])/len(sents)
Median = pd.Series([num.median(o) for o in Occurence])/len(sents)
Sdev = pd.Series([num.std(o) for o in Occurence])/len(sents)
Conceptdata = pd.DataFrame({'Concept':Concept,'Occurence':Occurence,'Frequency':Frequency,'Mean':Mean,'Median':Median,'Sdev':Sdev})

In [20]:
Conceptdata.sort_values(by='Frequency',ascending=False).head(200)

,Concept,Occurence,Frequency,Mean,Median,Sdev
193,parent,"[85, 113, 134, 136, 144, 163, 213, 214, 217, 2...",178,0.409674,0.416066,0.268445
530,child,"[113, 116, 143, 147, 203, 204, 205, 206, 208, ...",172,0.366698,0.286937,0.260058
220,critic,"[87, 118, 434, 435, 436, 469, 513, 515, 554, 6...",141,0.572694,0.567568,0.200868
20,flashback,"[1, 146, 150, 159, 161, 163, 173, 178, 191, 19...",139,0.477972,0.460661,0.242088
4,survivor,"[0, 95, 136, 147, 188, 191, 193, 199, 201, 295...",133,0.460261,0.457357,0.272234
349,fear,"[87, 97, 161, 173, 213, 214, 239, 250, 332, 34...",125,0.563133,0.686486,0.274024
94,feeling,"[30, 113, 172, 193, 201, 296, 355, 371, 385, 3...",104,0.529106,0.632883,0.304308
61,client,"[17, 101, 125, 389, 394, 519, 537, 587, 589, 5...",101,0.582511,0.703003,0.275223
28,life,"[4, 9, 30, 35, 38, 56, 66, 67, 149, 187, 188, ...",85,0.400512,0.413213,0.291423
313,recovery,"[87, 97, 320, 360, 504, 520, 536, 579, 593, 60...",79,0.443844,0.375976,0.274904


### Save as csv:

In [21]:
Conceptdata.to_csv('CPTSD'+'.csv',sep=',')

### Save dictionaries and dataframe to pickle file

In [22]:
import pickle
# filename = 'A Mind For Numbers_ How to Excel at Math and Science (Even If You Flunked Algebra)'
#filename = 'wikipedia4_wikihow3_no_wdlist'
#filename = 'classical_mech_wiki'
#filename = 'animal_kingdom_wiki'
filename= 'CPTSD'
concepts = {'sents':sents,'rawtxt':rawtxt,'sent_to_npflat':sent_to_npflat,'sent_to_tags':sent_to_tags,'sent_to_ltags':sent_to_ltags,'np_to_sent':np_to_sent,'Conceptdata':Conceptdata,'inv_eqn_dict':inv_eqn_dict}
with open('../processed_data/'+filename+'concepts.pickle', 'wb') as f:
    pickle.dump(concepts, f)
f.close()

In [23]:
filename

'CPTSD'

In [6]:
type(rawtxt)

str

In [9]:
sents[0:100]

['Title : Complex PTSD: From Surviving to Thriving: A GUIDE AND MAP FOR RECOVERING FROM CHILDHOOD TRAUMA\nAuthor: Walker, Pete\n\nGenerate From: eCore v0',
 '9',
 '12',
 '751 [ http://www',
 'epubor',
 'com/ecore',
 'html ]\n\nTechnology Website: http://www',
 'epubor',
 'com\n\n\n\n [image file=image8',
 'jpg]  [image file=image17',
 'jpg] Testimonials about Pete Walker’s first book, The Tao of Fully Feeling, and his website: www',
 'pete-walker',
 'comI am writing from Survivors of Abuse Recovering (S',
 'O',
 'A',
 'R',
 ') Society, located in Canada',
 ' We would like to include “13 Steps for Managing Flashbacks” in our resource manual',
 'I found myself',
 ' I found myself in your words',
 ' It’s as if you had unzipped me, stepped inside my traumatized inner self, meandered around a bit, come back outside, and wrote about what you discovered inside of me',
 ' For the first time in my life',
 '',
 '',
 '',
 '',
 '',
 '',
 'and I’m in my fifties now',
 '',
 '',
 '',
 '',
 '',
 '',
 